In [33]:
import pandas as pd
df_train = pd.read_csv('training_data.csv')
df_test =  pd.read_csv('test_data.csv')

In [36]:
df_train.columns

Index(['town', '#rooms', 'storey_range_processed', 'floor_area_sqm',
       'flat_model', 'remaining_lease', 'dist_nearest_bus_terminal',
       'bus_terminals_within_500m', 'bus_terminals_within_800m',
       'bus_terminals_within_1000m', 'dist_nearest_mrt',
       'dist_nearest_hawker_centers', 'dist_nearest_hospitals',
       'dist_nearest_mall', 'malls_within_3km', 'malls_within_5km',
       'dist_nearest_supermarkets', 'dist_nearest_schools',
       'dist_nearest_sport_facilities', 'resale_price', 'price_per_sqm'],
      dtype='object')

In [39]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
enc1 = LabelEncoder()
df_train['flat_model'] = enc1.fit_transform(df_train['flat_model'])
df_test['flat_model'] = enc1.transform(df_test['flat_model'])
enc2 = OneHotEncoder(handle_unknown='ignore')
enc_df = pd.DataFrame(enc2.fit_transform(df_train[['flat_model']]).toarray())
df_train = df_train.join(enc_df)
enc_df = pd.DataFrame(enc2.transform(df_test[['flat_model']]).toarray())
df_test = df_test.join(enc_df)

In [45]:
X = df_train.drop(columns=['town', 'flat_model','resale_price','price_per_sqm'])
y_1 = df_train['resale_price']
y_2 = df_train['price_per_sqm']
X_test = df_test.drop(columns=['town', 'flat_model','resale_price','price_per_sqm'])
y_test_1 = df_test['resale_price']
y_test_2 = df_test['price_per_sqm']

In [46]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold

regressor_1 = RandomForestRegressor(n_estimators=20, random_state=0)
regressor_1.fit(X, y_1)

# CV 
kfold = KFold(n_splits=5, shuffle=True)
kf_cv_scores = cross_val_score(regressor_1, X, y_1, cv=kfold, scoring='neg_mean_squared_error')
print("K-fold CV average MSE: %.2f" % kf_cv_scores.mean())

y_pred_1 = regressor_1.predict(X_test)

K-fold CV average MSE: -822688542.83


In [47]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_1, y_pred_1))
print('Mean Squared Error:', metrics.mean_squared_error(y_test_1, y_pred_1))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test_1, y_pred_1)))

Mean Absolute Error: 20122.597629150947
Mean Squared Error: 790168464.0317293
Root Mean Squared Error: 28109.935325996918


In [48]:
regressor_2 = RandomForestRegressor(n_estimators=20, random_state=0)
regressor_2.fit(X, y_2)

# CV 
kfold = KFold(n_splits=5, shuffle=True)
kf_cv_scores = cross_val_score(regressor_2, X, y_2, cv=kfold, scoring='neg_mean_squared_error')
print("K-fold CV average MSE: %.2f" % kf_cv_scores.mean())

y_pred_2 = regressor_2.predict(X_test)*X_test['floor_area_sqm']

K-fold CV average MSE: -82495.89


In [49]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_1, y_pred_2))
print('Mean Squared Error:', metrics.mean_squared_error(y_test_1, y_pred_2))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test_1, y_pred_2)))

Mean Absolute Error: 19793.604969235723
Mean Squared Error: 757120859.6108847
Root Mean Squared Error: 27515.8292553738
